In [153]:
import re
import numpy as np
import pandas as pd

In [154]:
import compress_fasttext
import fasttext

In [155]:
model = compress_fasttext.models.CompressedFastTextKeyedVectors.load(
    '/home/giovanni/unimore/TESI/src/models/fastText/cc.en.300.compressed.bin'
)

In [156]:
tokenization = fasttext.FastText.tokenize('That\'s my first trial with fasttext; it\s tricky!')
tokenization

["That's", 'my', 'first', 'trial', 'with', 'fasttext;', 'it\\s', 'tricky!']

# Embedding a Table

Now we try to embed every column of a single table:

through fastText, for every cell in a categorical column, we'll compute a tokenization and then a vectorization of the cell.

> cell --> token sequence --> token vector sequence --> cell vector

In [157]:
def my_tokenizer(s: str):
    return [re.sub('[^a-z]+', '', x) for x in s.lower().split()]

In [158]:
def compute_column_embeddings(df: pd.DataFrame):
    return \
        (
            np.mean( 
                np.array( 
                    [
                        np.sum( 
                            [model.get_vector(token) for token in cell], # compute cell embedding
                            axis=0)
                        for cell in df[column].apply(my_tokenizer)  # compute column embedding
                    ]
                ),
                axis=0
            ) 
        for column in df.columns
        )

### Comparing embeddings from UK football datasets - only categorical columns

In [159]:
stadium_df = pd.read_csv(
    '/home/giovanni/unimore/TESI/src/data/uk_football/List_of_football_stadiums_in_England_1.csv'
    ).drop(['Rank\n(England only)', 'Image', 'Capacity'], axis=1).dropna()

pl1_df = pd.read_csv(
    '/home/giovanni/unimore/TESI/src/data/uk_football/Premier_League_1.csv'
).drop('Capacity', axis=1).dropna()

In [160]:
stadium_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, 1 to 146
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Stadium      146 non-null    object
 1   Town / City  146 non-null    object
 2   Team         146 non-null    object
 3   League       146 non-null    object
dtypes: object(4)
memory usage: 5.7+ KB


In [161]:
stadium_df.head()

,Stadium,Town / City,Team,League
1,Old Trafford,"Old Trafford, Greater Manchester",Manchester United,Premier League
2,Tottenham Hotspur Stadium,"Tottenham, London",Tottenham Hotspur,Premier League
3,London Stadium,"Stratford, London",West Ham United,Premier League
4,Anfield,"Anfield, Liverpool",Liverpool,Premier League
5,Emirates Stadium,"Holloway, London",Arsenal,Premier League


In [162]:
pl1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Team      20 non-null     object
 1   Location  20 non-null     object
 2   Stadium   20 non-null     object
dtypes: object(3)
memory usage: 608.0+ bytes


In [163]:
pl1_df.head()

,Team,Location,Stadium
0,Arsenal,London (Holloway),Emirates Stadium
1,Aston Villa,Birmingham,Villa Park
2,Bournemouth,Bournemouth,Vitality Stadium
3,Brentford,London (Brentford),Gtech Community Stadium
4,Brighton & Hove Albion,Brighton,American Express Stadium


In [164]:
stadium_df.shape, pl1_df.shape

((146, 4), (20, 3))

In [165]:
stadium_embeddings = list(compute_column_embeddings(stadium_df))
pl1_embeddings = list(compute_column_embeddings(pl1_df))

In [166]:
len(stadium_embeddings), len(stadium_embeddings[0]), len(pl1_embeddings), len(pl1_embeddings[0])

(4, 300, 3, 300)

In [167]:
# TODO what is the best manner to store/process cell embeddings? Since they are needed both
# for row and column embeddings, it would be better to create a sort of cell-embedding-matrix and 
# then operate on it

In [168]:
def compare_datasets(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    emb1, emb2 = list(compute_column_embeddings(df1)), list(compute_column_embeddings(df2))
    comparisons = pd.DataFrame(columns=['C1', 'C2', 'cosine similarity'])
    for i, es in enumerate(df1.columns):
        for j, ep in enumerate(df2.columns):
            ei = emb1[i]
            ej = emb2[j]

            cosim = np.dot(ei, ej) / (np.linalg.norm(ei) * np.linalg.norm(ej))

            comparisons.loc[len(comparisons)] = [es, ep, cosim]

    return comparisons

In [169]:
compare_datasets(stadium_df, pl1_df)

,C1,C2,cosine similarity
0,Stadium,Team,0.672407
1,Stadium,Location,0.516966
2,Stadium,Stadium,0.967060
3,Town / City,Team,0.793410
4,Town / City,Location,0.954044
5,Town / City,Stadium,0.561669
6,Team,Team,0.917976
7,Team,Location,0.726615
8,Team,Stadium,0.743280
9,League,Team,0.503647


### Compare complete tables

In [170]:
# TODO how to handle unknown columns data types? 'Capacity' in stadium dataframe has the ',' character 
# in its values, and so pandas doesn't recognize it as a float

In [184]:
stadium_df = pd.read_csv(
    '/home/giovanni/unimore/TESI/src/data/uk_football/List_of_football_stadiums_in_England_1.csv'
    ).drop(['Rank\n(England only)', 'Image'], axis=1).dropna()

pl1_df = pd.read_csv(
    '/home/giovanni/unimore/TESI/src/data/uk_football/Premier_League_1.csv'
    ).dropna()

In [181]:
stadium_df.convert_dtypes().info()

<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, 1 to 146
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Stadium      146 non-null    string
 1   Town / City  146 non-null    string
 2   Capacity     146 non-null    string
 3   Team         146 non-null    string
 4   League       146 non-null    string
dtypes: string(5)
memory usage: 6.8 KB


In [182]:
pl1_df.convert_dtypes().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Team      20 non-null     string
 1   Location  20 non-null     string
 2   Stadium   20 non-null     string
 3   Capacity  20 non-null     string
dtypes: string(4)
memory usage: 768.0 bytes


In [183]:
compare_datasets(stadium_df, pl1_df)

/tmp/ipykernel_19381/2641501262.py:9: RuntimeWarning: invalid value encountered in scalar divide
  cosim = np.dot(ei, ej) / (np.linalg.norm(ei) * np.linalg.norm(ej))
/tmp/ipykernel_19381/2641501262.py:9: RuntimeWarning: invalid value encountered in scalar divide
  cosim = np.dot(ei, ej) / (np.linalg.norm(ei) * np.linalg.norm(ej))
/tmp/ipykernel_19381/2641501262.py:9: RuntimeWarning: invalid value encountered in scalar divide
  cosim = np.dot(ei, ej) / (np.linalg.norm(ei) * np.linalg.norm(ej))
/tmp/ipykernel_19381/2641501262.py:9: RuntimeWarning: invalid value encountered in scalar divide
  cosim = np.dot(ei, ej) / (np.linalg.norm(ei) * np.linalg.norm(ej))
/tmp/ipykernel_19381/2641501262.py:9: RuntimeWarning: invalid value encountered in scalar divide
  cosim = np.dot(ei, ej) / (np.linalg.norm(ei) * np.linalg.norm(ej))
/tmp/ipykernel_19381/2641501262.py:9: RuntimeWarning: invalid value encountered in scalar divide
  cosim = np.dot(ei, ej) / (np.linalg.norm(ei) * np.linalg.norm(ej))
/tmp

,C1,C2,cosine similarity
0,Stadium,Team,0.672407
1,Stadium,Location,0.516966
2,Stadium,Stadium,0.967060
3,Stadium,Capacity,NaN
4,Town / City,Team,0.793410
5,Town / City,Location,0.954044
6,Town / City,Stadium,0.561669
7,Town / City,Capacity,NaN
8,Capacity,Team,NaN
9,Capacity,Location,NaN


Without any kind of column preprocessing we have in output a lot of NaN values...